In [ ]:
import torch
import cv2
import numpy as np
from segment_anything import SamPredictor, sam_model_registry
# Load the pre-trained SAM model (choose the appropriate model variant)
sam = sam_model_registry["vit_h"](checkpoint="../sam_vit_h_4b8939.pth")
sam.to(device="cuda" if torch.cuda.is_available() else "cpu")
# Initialize the predictor
predictor = SamPredictor(sam)
# Load the image
image = cv2.imread("path_to_your_image.jpg")
predictor.set_image(image)
# Define a point (x, y) on the object to segment
input_point = np.array([[500, 500]])  # Example point coordinates
input_label = np.array([1])  # 1 indicates the object, 0 indicates the background
# Predict the mask based on the point
masks, _, _ = predictor.predict(point_coords=input_point, point_labels=input_label, multimask_output=False)
# Extract the first mask
mask = masks[0]
# Convert mask to binary (0 or 255)
binary_mask = (mask * 255).astype(np.uint8)
# Find contours (spatial information)
contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
for contour in contours:
   # Bounding box
   x, y, w, h = cv2.boundingRect(contour)
   print(f"Bounding Box: x={x}, y={y}, width={w}, height={h}")
   # Centroid
   M = cv2.moments(contour)
   if M["m00"] != 0:
       cX = int(M["m10"] / M["m00"])
       cY = int(M["m01"] / M["m00"])
       print(f"Centroid: x={cX}, y={cY}")
   # Optionally, draw the contour and bounding box on the original image for visualization
   cv2.drawContours(image, [contour], -1, (0, 255, 0), 2)
   cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
# Display the image with spatial information
cv2.imshow("Image with Spatial Information", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
# Save the image with drawn contours and bounding boxes (optional)
cv2.imwrite("output_with_spatial_info.png", image)